In [1]:
import os
from dotenv import load_dotenv 
load_dotenv(override=True)

True

## Model I/O 模块

In [9]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chat_models import init_chat_model
from langchain_core.output_parsers import StrOutputParser

# 1. 定义提示词模板
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个专业的AI助手。"),
    ("user", "{input}")
])

# 2. 初始化模型 - 请根据DeepSeek的实际安装包替换ChatDeepSeek
llm = init_chat_model(model="deepseek-chat", temperature=0.7)

# 3. 定义输出解析器
output_parser = StrOutputParser()

# 4. 使用LCEL组合成链
chain = prompt | llm | output_parser



In [10]:
# 调用链
response = chain.invoke({"input": "请解释一下什么是机器学习。"})
print(response)

当然可以！我很乐意为你解释什么是机器学习。

这是一个可以用一句话概括的核心定义：

**机器学习是人工智能的一个分支，其核心是让计算机通过“学习”数据中的规律和模式，从而获得预测或决策的能力，而无需进行显式的编程。**

下面我们来详细拆解这个定义，让它更容易理解：

### 1. 核心思想：从“编程”到“学习”

*   **传统编程**：我们给计算机输入**规则（程序）** 和**数据**，计算机输出**答案**。
    *   例如：我们写一个程序来识别猫，规则可能是“如果有胡须、尖耳朵、长尾巴，那就是猫”。这个规则需要程序员自己总结并精确地写出来。
*   **机器学习**：我们给计算机输入**数据**和对应的**答案**，计算机自己找出连接数据和答案的**规则（模型）**。
    *   例如：我们给计算机看成千上万张“猫”和“非猫”的图片（数据+答案），计算机通过算法自己总结出猫的特征（比如毛茸茸的纹理、眼睛的形状等），最终形成一个可以识别新图片中是否有猫的模型。

所以，机器学习的本质是 **“让机器自己从数据中学习规则”**。

### 2. 一个简单的类比：教孩子认动物

想象一下你教一个孩子认识“狗”：
1.  你给他看很多不同品种、大小、颜色的狗的图片，并告诉他“这是狗”。
2.  你也给他看一些猫、马、汽车的图片，并告诉他“这不是狗”。
3.  经过一段时间的学习，孩子的大脑会自己总结出“狗”的关键特征（比如四条腿、摇尾巴、汪汪叫等）。
4.  之后，当他看到一只他从未见过的狗时，他也能正确地认出那是狗。

**机器学习的过程与此非常相似：**
*   **数据** = 你给孩子看的各种图片。
*   **答案（标签）** = 你告诉孩子“是狗”或“不是狗”。
*   **学习算法** = 孩子大脑总结规律的过程。
*   **模型** = 孩子大脑中形成的关于“狗”的概念。
*   **预测** = 孩子识别新动物。

### 3. 机器学习的三种主要类型

根据学习方式的不同，机器学习主要分为三类：

1.  **监督学习**
    *   **特点**：数据是**带标签的**。就像有标准答案的学习资料。
    *   **目标**：学习输入到输出之间的映射关系，用于**预测**。
    *   **例子**：
       

## Retrieval 

In [12]:
from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.embeddings import init_embeddings
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.retrievers import EnsembleRetriever
from langchain.retrievers import BM25Retriever
from langchain_core.documents import Document

# 1. 文档加载与处理
docs = [
    "LangChain 将 LLM、工具、检索串成可观测工作流。",
    "LangGraph 用状态图编排多步骤/多智能体流程，适合生产级代理。",
    "检索增强生成(RAG)通过结合检索器和生成模型来提高回答质量。",
    "向量数据库用于高效存储和检索嵌入向量，支持相似性搜索。"
]

# 2. 文本分割 - 优化参数
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=120,
    chunk_overlap=20,
    separators=["\n\n", "\n", "。", "！", "？", "；", "，", " ", ""]  # 中文友好的分隔符
)
splits = text_splitter.create_documents(docs)

# 3. 嵌入模型初始化
emb = init_embeddings(
    "openai:text-embedding-3-small", 
    openai_api_key="sk-cRtxv5U9AdK45kVNr4InXs3oIUTnnb6L3H5xJ9AaAA7qUe53", 
    openai_api_base="https://chatapi.littlewheat.com/v1"
)

# 4. 向量存储与检索器
vectordb = FAISS.from_documents(splits, emb)
base_retriever = vectordb.as_retriever(
    search_type="similarity",  # 可选 "similarity", "mmr", "similarity_score_threshold"
    search_kwargs={"k": 3}  # 返回前3个最相关文档
)

# 5. 高级检索技术：上下文压缩 :cite[10]
llm = init_chat_model("deepseek:deepseek-chat")
compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=base_retriever
)

# 6. 混合检索策略（语义 + 关键词）:cite[10]
bm25_retriever = BM25Retriever.from_documents(splits)
bm25_retriever.k = 3

ensemble_retriever = EnsembleRetriever(
    retrievers=[base_retriever, bm25_retriever],
    weights=[0.7, 0.3]  # 语义检索权重更高
)

# 7. RAG链构建
def format_docs(docs):
    """格式化检索到的文档"""
    return "\n\n".join(f"• {doc.page_content}" for doc in docs)

prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个专业的技术助手。请严格依据以下上下文信息回答问题。\n\n上下文：{context}\n\n要求：如果上下文信息不足以回答问题，请明确说'根据已知信息无法完整回答这个问题。'"),
    ("human", "问题：{question}")
])

# 使用LCEL构建RAG链 :cite[1]:cite[3]
rag_chain = (
    {"context": compression_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 8. 测试不同检索器效果
def test_retrieval_systems(question):
    """测试不同检索系统的效果"""
    print(f"问题: {question}")
    
    # 基础向量检索
    basic_docs = base_retriever.invoke(question)
    print(f"\n基础检索结果数: {len(basic_docs)}")
    
    # 压缩检索
    compressed_docs = compression_retriever.invoke(question)
    print(f"压缩检索结果数: {len(compressed_docs)}")
    
    # 混合检索
    ensemble_docs = ensemble_retriever.invoke(question)
    print(f"混合检索结果数: {len(ensemble_docs)}")
    
    # 生成回答
    response = rag_chain.invoke(question)
    print(f"\n回答: {response}")
    print("=" * 50)


# 9. 持久化向量存储（可选）
# vectordb.save_local("faiss_index")
# 加载时使用: vectordb = FAISS.load_local("faiss_index", emb, allow_dangerous_deserialization=True)

In [13]:
# 测试示例
test_questions = [
    "LangGraph 适合哪些场景？",
    "什么是检索增强生成？",
    "LangChain 的主要功能是什么？"
]

for question in test_questions:
    test_retrieval_systems(question)

问题: LangGraph 适合哪些场景？

基础检索结果数: 3
压缩检索结果数: 1
混合检索结果数: 4

回答: 根据已知信息，LangGraph 适合用于编排多步骤或多智能体的流程，尤其适用于生产级代理场景。
问题: 什么是检索增强生成？

基础检索结果数: 3
压缩检索结果数: 1
混合检索结果数: 4

回答: 根据已知信息，检索增强生成（RAG）是一种通过结合检索器和生成模型来提高回答质量的技术。
问题: LangChain 的主要功能是什么？

基础检索结果数: 3
压缩检索结果数: 1
混合检索结果数: 4

回答: 根据已知信息，LangChain 的主要功能是将 LLM（大语言模型）、工具和检索串成可观测的工作流。


## Memory 

In [16]:
from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain.memory import ChatMessageHistory
from langchain_core.runnables import RunnablePassthrough

# 初始化模型
llm = init_chat_model("deepseek:deepseek-chat")

# 创建对话记忆存储
message_history = ChatMessageHistory()

# 构建带有记忆的提示模板
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个友好的AI助手，能够记住对话历史并与人进行连贯的对话。"),
    MessagesPlaceholder(variable_name="history"),  # 这里会注入历史消息
    ("human", "{input}")
])

# 创建对话链
conversation_chain = (
    {
        "history": lambda x: message_history.messages,  # 获取历史消息
        "input": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

def chat_with_memory(user_input):
    """带有记忆的对话函数"""
    # 获取AI回复
    response = conversation_chain.invoke(user_input)
    
    # 将用户输入和AI回复都保存到历史中
    message_history.add_user_message(user_input)
    message_history.add_ai_message(response)
    
    return response

# 测试对话
print("AI: 你好！我是您的AI助手，我可以记住我们的对话。您叫什么名字？")


AI: 你好！我是您的AI助手，我可以记住我们的对话。您叫什么名字？


In [17]:
# 第一轮对话
user_input1 = "我叫小明"
response1 = chat_with_memory(user_input1)
print(f"用户: {user_input1}")
print(f"AI: {response1}")

# 第二轮对话 - AI会记得名字
user_input2 = "你还记得我的名字吗？"
response2 = chat_with_memory(user_input2)
print(f"用户: {user_input2}")
print(f"AI: {response2}")

# 查看记忆内容
print(f"\n当前对话历史:")
for i, message in enumerate(message_history.messages):
    role = "用户" if message.type == "human" else "AI"
    print(f"{role}: {message.content}")

用户: 我叫小明
AI: 你好，小明！很高兴认识你。今天有什么我可以帮你的吗？无论是学习、生活还是其他问题，我都很乐意和你聊聊！😊
用户: 你还记得我的名字吗？
AI: 当然记得！你刚才告诉我你叫小明。我会认真记住我们的对话内容，以便更好地帮助你。有什么想聊的吗？😄

当前对话历史:
用户: 我叫小明
AI: 你好，小明！很高兴认识你。今天有什么我可以帮你的吗？无论是学习、生活还是其他问题，我都很乐意和你聊聊！😊
用户: 你还记得我的名字吗？
AI: 当然记得！你刚才告诉我你叫小明。我会认真记住我们的对话内容，以便更好地帮助你。有什么想聊的吗？😄


### 高级封装

In [18]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

# 使用内置的ConversationBufferMemory
memory = ConversationBufferMemory(return_messages=True)

# 创建对话链
conversation = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=False  # 设为True可以看到详细的执行过程
)

# 进行多轮对话
print("=== 多轮对话演示 ===")

dialogue = [
    "你好，我叫李华",
    "我今年25岁，是一名软件工程师",
    "你还记得我的名字和职业吗？",
    "我最喜欢的编程语言是Python",
    "总结一下你了解的关于我的信息"
]

for user_input in dialogue:
    response = conversation.predict(input=user_input)
    print(f"用户: {user_input}")
    print(f"AI: {response}")
    print("-" * 40)

# 查看完整的记忆内容
print("\n📝 完整记忆内容:")
print(memory.buffer)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_11888\2814691052.py:5: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(return_messages=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_11888\2814691052.py:8: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  conversation = ConversationChain(


=== 多轮对话演示 ===
用户: 你好，我叫李华
AI: 你好李华！很高兴认识你！😊 我是一个AI助手，可以和你聊天、回答问题、帮助你解决各种问题。你今天过得怎么样？有什么想聊的话题或者需要我帮助的地方吗？比如我们可以聊聊天气、生活、学习、工作，或者任何你感兴趣的事情！
----------------------------------------
用户: 我今年25岁，是一名软件工程师
AI: 太棒了！25岁的软件工程师，这正是一个充满活力和发展潜力的阶段！😊

作为软件工程师，你的工作一定很有趣也很有挑战性吧？不知道你主要专注于哪个技术栈呢？是前端开发、后端架构、移动端开发，还是全栈？

我猜你的日常可能涉及到：
- 编写和调试代码
- 参与技术方案设计
- 与团队成员协作开发
- 学习新的技术和框架

这个行业变化很快，需要持续学习，但同时也带来了很多成长机会。你现在在做什么有趣的项目吗？或者最近在学习和探索什么新技术？

我很想听听你在工作中的一些经历和感受！毕竟软件工程不仅仅是写代码，还涉及到解决问题、创造价值，这些都是很有意思的话题！💻✨
----------------------------------------
用户: 你还记得我的名字和职业吗？
AI: 当然记得！你叫李华，今年25岁，是一名软件工程师。我们刚才还聊到你的工作内容和可能涉及的技术领域呢！😊  
作为软件工程师，你最近在忙什么有趣的项目吗？或者有没有遇到什么特别的技术挑战想分享一下？我很乐意听听你的故事～ 💻✨
----------------------------------------
用户: 我最喜欢的编程语言是Python
AI: 太棒了！Python是一门非常优秀的编程语言！🐍 作为软件工程师，选择Python真的很明智，它既强大又灵活。

Python的魅力在于它的：
- **简洁优雅的语法** - 让代码更易读易写
- **丰富的生态系统** - Django、Flask、NumPy、Pandas等强大的库
- **多领域应用** - Web开发、数据分析、人工智能、自动化脚本等
- **强大的社区支持** - 遇到问题很容易找到解决方案

我猜你喜欢Python可能是因为：
✨ 开发效率高，快速实现想法
✨ 代码可读性好，维护起来相对轻松
✨ 在机器学习和数

## Chains 

In [19]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# 1. 定义一个更复杂的提示词模板，包含上下文和历史
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "你是一个乐于助人的助手。根据以下上下文和对话历史回答问题。\n\n上下文：{context}"),
    MessagesPlaceholder(variable_name="history"),
    ("user", "{question}")
])

# 2. 构建一个模拟的检索器（这里用固定文本代替，实际可接入向量数据库）
def mock_retriever(question: str):
    # 模拟根据问题返回相关上下文
    return "这是一个模拟的上下文：机器学习是人工智能的一个分支。"

# 3. 使用LCEL组合一个包含检索和记忆的复杂链
chain = (
    {
        "context": lambda x: mock_retriever(x["question"]),
        "question": RunnablePassthrough(),
        "history": lambda x: x.get("history", [])  # 从输入中获取历史
    }
    | prompt_template
    | llm  # 使用之前定义的llm
    | StrOutputParser()
)

# 假设我们有一段历史
history_messages = []  # 这里可以填入之前的对话消息

In [20]:
# 调用链
result = chain.invoke({
    "question": "机器学习是什么？",
    "history": history_messages
})
print(result)

机器学习是人工智能的一个分支，专注于开发算法和统计模型，使计算机系统能够通过数据学习和改进，而无需显式编程。


## Agents 

In [24]:
from langchain_core.tools import tool
# 注意：Agent相关的Executor可能在langchain_experimental中，请关注官方更新

# 定义工具
@tool
def custom_calculator(input_text):
    """用于执行数学计算"""
    try:
        # 安全地评估数学表达式
        result = eval(input_text)
        return f"计算结果: {result}"
    except:
        return "计算失败，请检查表达式"


# 初始化代理 - 具体API可能随版本调整，建议查阅最新文档
tools = [custom_calculator]

# 将工具绑定到模型
llm_with_tools = llm.bind_tools(tools)

# 运行代理
# agent.run("计算15的平方是多少？")

In [25]:
print(custom_calculator.name)
print(custom_calculator.description)
print(custom_calculator.args)

custom_calculator
用于执行数学计算
{'input_text': {'title': 'Input Text'}}


In [26]:
response = llm_with_tools.invoke("计算15的平方是多少？")

print(response)

content='我来帮您计算15的平方。' additional_kwargs={'tool_calls': [{'id': 'call_00_E5TmqCLMaLUI5y0gRViLzgPq', 'function': {'arguments': '{"input_text": "15^2"}', 'name': 'custom_calculator'}, 'type': 'function', 'index': 0}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 149, 'total_tokens': 175, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 149}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_ffc7281d48_prod0820_fp8_kvcache', 'id': '3ea6e024-6310-4135-9892-76f22aa5ff9f', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None} id='run--60666dfe-aa3c-41c4-9f87-71c579123a9f-0' tool_calls=[{'name': 'custom_calculator', 'args': {'input_text': '15^2'}, 'id': 'call_00_E5TmqCLMaLUI5y0gRViLzgPq', 'type': 'tool_call'}] usage_metadata={'input_tokens': 149, 'output_tokens': 26, 'total_tokens': 175, 'input_token_details':

In [27]:
response.additional_kwargs

{'tool_calls': [{'id': 'call_00_E5TmqCLMaLUI5y0gRViLzgPq',
   'function': {'arguments': '{"input_text": "15^2"}',
    'name': 'custom_calculator'},
   'type': 'function',
   'index': 0}],
 'refusal': None}

## Callbacks 

In [46]:
import asyncio

prompt = ChatPromptTemplate.from_messages([
    ("system", "仅依据给定{context}回答；若缺少依据请说“我不知道”。"),
    ("human", "{question}")  # 模板期望一个名为 'question' 的变量
])
# 使用LCEL链的astream_log方法监控执行过程
async def monitor_chain():
    chain = prompt | llm | StrOutputParser()
    # 在输入字典中同时提供 'context' 和 'question'
    async for event in chain.astream_log({
        "context": "这是一个模拟的上下文。",  # 为 context 提供模拟值
        "question": "请写一首关于秋天的短诗。"
    }, include_names=["ChatDeepSeek"]):
        print(f"Event: {event}")

In [47]:
await monitor_chain()

Event: RunLogPatch({'op': 'replace',
  'path': '',
  'value': {'final_output': None,
            'id': 'a045b1dc-d2c7-4f3a-8425-d9c689a2f9f3',
            'logs': {},
            'name': 'RunnableSequence',
            'streamed_output': [],
            'type': 'chain'}})
模型开始运行
Event: RunLogPatch({'op': 'add',
  'path': '/logs/ChatDeepSeek',
  'value': {'end_time': None,
            'final_output': None,
            'id': '10f79ad6-e9ab-46c7-bf7d-399bed07e613',
            'metadata': {'ls_model_name': 'deepseek-chat',
                         'ls_model_type': 'chat',
                         'ls_provider': 'openai',
                         'ls_temperature': None},
            'name': 'ChatDeepSeek',
            'start_time': '2025-10-11T06:39:55.444+00:00',
            'streamed_output': [],
            'streamed_output_str': [],
            'tags': ['seq:step:2'],
            'type': 'llm'}})


Event: RunLogPatch({'op': 'add', 'path': '/logs/ChatDeepSeek/streamed_output_str/-', 'value': ''},
 {'op': 'add',
  'path': '/logs/ChatDeepSeek/streamed_output/-',
  'value': AIMessageChunk(content='', additional_kwargs={}, response_metadata={}, id='run--10f79ad6-e9ab-46c7-bf7d-399bed07e613')})
Event: RunLogPatch({'op': 'add', 'path': '/streamed_output/-', 'value': ''},
 {'op': 'replace', 'path': '/final_output', 'value': ''})
Event: RunLogPatch({'op': 'add',
  'path': '/logs/ChatDeepSeek/streamed_output_str/-',
  'value': '依据'},
 {'op': 'add',
  'path': '/logs/ChatDeepSeek/streamed_output/-',
  'value': AIMessageChunk(content='依据', additional_kwargs={}, response_metadata={}, id='run--10f79ad6-e9ab-46c7-bf7d-399bed07e613')})
Event: RunLogPatch({'op': 'add', 'path': '/streamed_output/-', 'value': '依据'},
 {'op': 'replace', 'path': '/final_output', 'value': '依据'})
Event: RunLogPatch({'op': 'add',
  'path': '/logs/ChatDeepSeek/streamed_output_str/-',
  'value': '给定的'},
 {'op': 'add',
  'pa